In [ ]:
import sys
import os

current_dir = os.getcwd()

project_root = os.path.abspath(os.path.join(os.path.dirname(current_dir), '.'))
if project_root not in sys.path:
    sys.path.append(project_root)

import pinns

# For cleaner output.
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

We will solve problem from https://arxiv.org/abs/2007.14527, section 7.2, with fabricated solution:

Find such $u(x)$ so that

$$\frac{\partial^2 u}{\partial x^2} = -16\pi^2 \sin(4\pi x) \quad \text{on} \quad [0, 1]$$
$$u(0) = u(1) = 0$$

And solution reads as:

$$u(x) = \sin(4\pi x)$$

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
x = torch.linspace(0, 1, 100)
u = torch.sin(4*torch.pi * x)

plt.plot(x, u)
plt.grid()
plt.show()

In [ ]:
from pinns.samplers import ConstantSampler, RandomRectangularSampler

pts = torch.tensor([x[0], x[-1]]).reshape(-1, 1)
vals = torch.tensor([u[0], u[-1]]).reshape(-1, 1)
constraints_sampler = ConstantSampler((pts, vals))

domain = {'x': [0, 1]}
collocation_sampler = RandomRectangularSampler(domain, 256, return_dict=False)

test_points_sampler = ConstantSampler((
    x.reshape(-1, 1), u.reshape(-1, 1)
))

In [ ]:
from pinns.derivatives import Derivative

d = Derivative(method = 'autograd')

def loss(
    cstr_pts, cstr_pred, cstr_vals,
    coll_pts, coll_pred
    ):
    
    def boundary_loss(u, x):
        return torch.mean(torch.square(u - cstr_vals))

    def ode_loss(u, x):
        uxx = d(u, x, orders = 2)
        return torch.mean(torch.square(uxx + 16*torch.pi**2 * torch.sin(4*torch.pi*x)))
    
    losses = (
        boundary_loss(cstr_pred, cstr_pts),
        ode_loss(coll_pred, coll_pts)
    )
    
    return losses

In [ ]:
from pinns import Trainer
from pinns.models import FF
from pinns.optimizers import Adam

pinn = FF([1] + [64] + [1], activ=nn.Tanh(), biases=True)
print(f'Model has {pinn.count_parameters()} trainable parameters.')

adam = Adam(pinn, lr = 1e-2)

trainer = Trainer(
    loss,
    pinn,
    constraints_sampler,
    collocation_sampler,
    loss_coefs=[0.8, 0.2],    # Coefficients are very important.
    test_points_sampler=test_points_sampler
)

num_iters = 1500
save_every = 10

def make_plot():
    if trainer.iter == 0 or trainer.iter % save_every == 0 or trainer.iter == num_iters:
        preds = pinn.predict(test_points_sampler()[0]).detach()
        np.save(f'./.temp/poisson_{trainer.iter}.npy', preds.numpy())

trainer.train(
    num_iters=num_iters,
    optimizers=[(0, adam)],
    validate_every=1,
    at_training_start_callbacks=[make_plot],
    at_epoch_end_callbacks=[make_plot],
    at_training_end_callbacks=[make_plot]
    )

In [ ]:
# pinn.model = torch.load('./very_good_model_dont_delete.pt')

fig, axs = plt.subplots(1, 2, figsize=(10, 3))

axs[0].plot(trainer.loss_history, label='Loss')
axs[0].plot(range(0, trainer.iter + 1, 1), trainer.error_history, label='L2')
axs[0].grid()
axs[0].set_yscale('log')
axs[0].legend()

preds = pinn.predict(x.reshape(-1, 1))
axs[1].plot(x, u, label='Solution')
axs[1].plot(x, preds.detach(), label='Predicts', linestyle=':')
axs[1].grid()
axs[1].legend()

plt.show()

In [ ]:
from PIL import Image
import imageio
from joblib import Parallel, delayed

from tqdm.notebook import tqdm_notebook as tqdm

def save_animation(files, path, duration=5, fps=60, loop=0, type='mp4', processors=2, ):
    
    fig = plt.figure(figsize=(5, 3))
    xlim = x.min(), x.max()
    
    def plot(i):
        predictions = np.load(files[i])
        # Set plot limits and labels
        plt.xlim(xlim)
        plt.plot(x, u, label='Solution')
        plt.plot(x, predictions, label='Predicts', linestyle=':')
        plt.grid()
        plt.legend()
        fig.savefig(f'./.temp/frame_{i}.png', dpi=300)
        fig.clear()
        
    # Number of frames
    num_frames = len(files)

    # Parallelize the plotting function
    Parallel(n_jobs=processors, verbose=4)(delayed(plot)(i) for i in range(num_frames))
    
    if type == 'mp4':
        writer = imageio.get_writer(path, fps=fps)
        for i in range(len(files)):
            writer.append_data(imageio.imread(f'./.temp/frame_{i}.png'))
        writer.close()
        
    if type == 'gif':
        imgs = [Image.open(f'./.temp/frame_{i}.png') for i in range(len(files))]
        imgs[0].save(path, save_all=True, append_images=imgs[1:], duration=duration, fps=fps, loop=loop)
    
files = [f'./.temp/poisson_{i}.npy' for i in range(0, trainer.iter, save_every)]
save_animation(files, './.results/poisson animation.gif', type='gif', processors=8)